In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip3 install --quiet albumentations

In [ ]:
package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master', '../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0',
    '../input/image-fmix/FMix-master', '../input/pretrainedmodels/pretrainedmodels-0.7.4'
]

import sys
sys.path.extend(package_paths)

In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss
import json
import seaborn as sns
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import defaultdict
import torch.nn.functional as F
import pretrainedmodels

In [ ]:
# Import for albumentations
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

In [ ]:
CFG = {
    'input_dir': "/kaggle/input/cassava-leaf-disease-classification",
    'train_folder': "train_images",
    'test_folder': "test_images",
    'seed': 42,
    'model_arch': ['tf_efficientnet_b5_ns', 'tf_efficientnet_b4_ns', 'tf_efficientnet_b4_ns', 'tf_efficientnet_b5_ns', 'resnet200d',
                    'resnext50_32x4d', 'seresnext50_32x4d', 'seresnext50_32x4d', 'vit_base_patch16_384', 'se_resnext101_32x4d', 'seresnext50_32x4d', 'tf_efficientnet_b5_ns'],
    'model_paths': ['/kaggle/input/single-model-efficient-net-b5-ns-wo-dropout', '/kaggle/input/single-model-efficientnet-b4-ns-w-dropout',
                    '/kaggle/input/single-model-efficientnet-b4-ns', '/kaggle/input/single-model-efficientnet-b5-ns',
                    '/kaggle/input/single-model-resnet200d', '/kaggle/input/single-model-resnext50', '/kaggle/input/single-model-seresnext50', 
                    '/kaggle/input/single-model-seresnext50-wo-dropout', '/kaggle/input/single-model-vision-transformer', '/kaggle/input/single-model-seresnext101',
                    '/kaggle/input/single-model-seresnext50-kd-and-pl', '/kaggle/input/single-model-efficientnet-b5-ns-kd-and-pl'],
    'name_model': ['EfficientNet b5 ns wo dropout', 'EfficientNet b4 ns', 'EfficientNet b4 ns wo dropout', 'EfficientNet b5 ns', 'ResNet200d',
                   'ResNeXt50', 'SEResNeXt50', 'SEResNeXt50 wo dropout', 'Vision Transformer', 'SEResNeXt101', 'SEResNext50 w kd and pl', 'EfficientNet b5 ns kd and pl'],
    'name_class': [],
    'img_size': 512,
    'verbose_step': 1,
    'device': 'cuda:0',
    'num_classes': 5,
    'num_folds': 5,
    'train_bs': 16,
    'valid_bs': 100,
    'test_bs': 100,
    'num_workers': 4,
    'tta': 1,
    'm': [0, 4, 10, 6, 1, 8],# 2],
    'w': [0.49, 0.2266, 0.092, 0.1948, 0.1238]#, 0.003]
}

In [ ]:
CFG['name_class'] = ['MyClassifier' + (str(2) if ('dropout' in name) else str(1))  for name in CFG['name_model'][:-3]] + ['MyClassifier3', 'MyClassifier1', 'MyClassifier1']

In [ ]:
info_df = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")
info_df.head()

In [ ]:
submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
submission.head()

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def rand_bbox(size, lam):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

In [ ]:
class CassavaLeafDataset(Dataset):
    def __init__(
        self, df, root_dir, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.root_dir = root_dir
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def read_img(self, image_id):
        path_img = os.path.join(CFG["input_dir"], self.root_dir, str(image_id))
        img = cv2.imread(path_img)[:,:,::-1]
        return img
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
        
        img  = self.read_img(self.df.iloc[index]['image_id'])
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"], float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )
    
    def __getitem__(self, metric_name):
        metric = self.metrics[metric_name]
        return metric["avg"]

In [ ]:
def do_nothing(image, *args, **kwargs):
    return {"image": image}

def enable_if(condition, obj):
    return obj if (condition) else do_nothing

def get_augmentation(img_size, mode='train'):
    return A.Compose([
        enable_if(mode == "train", A.RandomResizedCrop(img_size, img_size)),
        enable_if(mode == "train", A.Transpose(p=0.5)),
        enable_if(mode == "train", A.HorizontalFlip(p=0.5)),
        enable_if(mode == "train", A.VerticalFlip(p=0.5)),
        enable_if(mode == "train", A.ShiftScaleRotate(p=0.5)),
        enable_if(mode == "train", A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5)),
        enable_if(mode == "train", A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5)),
        enable_if(mode == "val", A.CenterCrop(img_size, img_size, p=1)),
        enable_if(mode == "val", A.Resize(img_size, img_size)),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        enable_if(mode == "train", A.CoarseDropout(p=0.5)),
        enable_if(mode == "train", A.Cutout(p=0.5)),
        ToTensorV2(p=1.0),
    ])

def get_inference_augmentation(img_size):
    return Compose([
            RandomResizedCrop(img_size, img_size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [ ]:
class MyClassifier1(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        last_linear = None
        if ('vit' in model_arch):
            n_features = self.model.head.in_features
            self.model.head = nn.Sequential(
                                        nn.Dropout(0.3),
                                        nn.Linear(n_features, num_classes)
                                    )
        elif ('tf_efficientnet' in model_arch):
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(
                                        nn.Dropout(0.3),
                                        nn.Linear(n_features, num_classes)
                                    )
        else:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Sequential(
                                        nn.Dropout(0.3),
                                        nn.Linear(n_features, num_classes)
                                    )
    def forward(self, x):
        return self.model(x)
    
class MyClassifier2(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        last_linear = None
        if ('vit' in model_arch):
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, num_classes)
        elif ('tf_efficientnet' in model_arch):
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, num_classes)
        else:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, num_classes)
    def forward(self, x):
        return self.model(x)
    
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, x):
        return x.view(x.shape[0], -1)

class MyClassifier3(nn.Module):
    def __init__(self, model_arch, num_classes, pretrained=False):
        super().__init__()
        self.model = nn.Sequential(
            *(list(pretrainedmodels.__dict__['se_resnext101_32x4d'](num_classes=1000, pretrained=None).children())[:-2]),
            nn.AdaptiveAvgPool2d((1, 1)),
            Flatten(),
            nn.Dropout(0.3),
            nn.Linear(2048, num_classes)
        )
    def forward(self, x):
        return self.model(x)

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
tf_path_model = "../input/cassava-leaf-disease-1st-place-models/cropnet_mobilenetv3/cropnet"
import tensorflow_hub as hub
import tensorflow as tf

def build_mobilenetv3(img_size=(224, 224), weights="../input/cassava-leaf-disease-1st-place-models/cropnet_mobilenetv3/cropnet"):
    classifier = hub.KerasLayer(weights)
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        hub.KerasLayer(classifier, trainable=False)
    ])
    return model

def read_preprocess_file(img_path, normalize=False):
    img = cv2.imread(img_path)[:,:,::-1]
    if (normalize):
        img = img/255.0
    img = img.astype(np.float32)
    return img.shape, img

def image_augmentations(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > 0.75:
        image = tf.image.transpose(image)
        
    if p_rotate > 0.75:
        image = tf.image.rot90(image, k = 3)
    elif p_rotate > 0.5:
        image = tf.image.rot90(image, k = 2)
    elif p_rotate > 0.25:
        image = tf.image.rot90(image, k = 1)

    image = tf.image.resize(image, size = IMAGE_SIZE)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

def cut_crop_image(processed_img):
    image = tf.image.central_crop(processed_img, 0.8)
    image = tf.image.resize(image, (224, 224))
    return np.expand_dims(image, 0)

def distribute_unknown(probabilities):
    return probabilities[:,:-1] + np.expand_dims(probabilities[:,-1]/5, axis=1)

def multi_predict_tfhublayer(img_path, modelinstance):
    img = cut_crop_image(read_preprocess_file(img_path, True)[1])
    yhat = modelinstance.predict(img)
    return np.mean(distribute_unknown(yhat), axis=0)

def predict_and_vote(image_list, modelinstances, onlykeras):
    predictions = [] 
    with tqdm(total=len(image_list)) as process_bar:       
        for img_path in image_list:
            process_bar.update(1)  
            Yhats = np.vstack([func(img_path, modelinstance) for func, modelinstance in modelinstances])
            if onlykeras:
                predictions.append(np.argmax(np.sum(Yhats, axis=0)))
            else:
                predictions.append(Yhats)
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [ ]:
def inference(test_df, test_dir=CFG["test_folder"]):
    device = torch.device(CFG["device"])
    
    tst_all = []
    for i, m in enumerate(CFG['m']):
        model_arch = CFG['model_arch'][m]
        img_size = 512 if ('vit' not in model_arch) else 384
        test_transforms = get_augmentation(img_size, mode="val")
        test_dataset = CassavaLeafDataset(test_df, test_dir, test_transforms, output_label=False)
        test_loader = DataLoader(test_dataset, batch_size=CFG["test_bs"], shuffle=False, num_workers=CFG["num_workers"])
        model = globals()[CFG['name_class'][m]](model_arch, CFG['num_classes']).to(device)
        paths = np.array(glob(CFG['model_paths'][m] + "/*"))
        fold_idx = np.array([int(path.split('fold_')[1][0]) for path in paths])
        paths = paths[np.argsort(fold_idx)]
        tst_preds_one_model = []
        for fold in range(5):
            model.load_state_dict(torch.load(paths[fold]))
            for _ in range(1):
                with torch.no_grad():
                    img_preds_all = inference_one_epoch(model, test_loader, device)
                    tst_preds_one_model += [img_preds_all]
        tst_preds_one_model = np.mean(tst_preds_one_model, axis=0)
        tst_all += [tst_preds_one_model]
    final_output = tst_all[0]
    for k in range(1, len(tst_all)):
        final_output = (1 - CFG["w"][k-1]) * final_output + CFG["w"][k-1] * tst_all[k]
    return final_output

In [ ]:
pytorch_prob = inference(submission)
model_mobilenet = build_mobilenetv3()
tensorflow_prob = predict_and_vote([os.path.join(CFG["input_dir"], CFG["test_folder"], id) for id in submission["image_id"].values], [(multi_predict_tfhublayer, model_mobilenet)], onlykeras=False)
submission["label"] = np.argmax(np.mean([pytorch_prob, tensorflow_prob], axis=0), axis=1)
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()